In [1]:
import requests
import gzip
import pandas as pd
import networkx as nx
import numpy as np
import cPickle
import sys
from collections import Counter
import scipy.sparse
from scipy.sparse import coo_matrix, csc_matrix, csr_matrix, isspmatrix, issparse, isspmatrix_csr, isspmatrix_csc

%load_ext line_profiler
%load_ext autoreload
%autoreload 2

sys.path.insert(0, '/cellar/users/mikeyu/DeepTranslate/ddot')
import ddot
from ddot import Ontology

import pmc

import clixov_utils
import clixov_merge
import clique_maximal
import clique_maxcover
import mkl_spgemm
from clixov_merge import *
from clixov_utils import *

In [2]:
np.set_printoptions(linewidth=150)

In [3]:
# import warnings
# warnings.filterwarnings("error")

In [4]:
np.set_printoptions(precision=2)

In [5]:
# import os
# os.environ['NUMBA_DISABLE_JIT'] = '1'

# Load semantic similarity

## Yeast GO

In [6]:
f = np.load('yeast_go_20aug2017/resnik_ss.npz')
ss, ss_nodes = f['ss'], f['ss_nodes']

ont = Ontology.from_table('yeast_go_20aug2017/collapsed_go.20aug2017', clixo_format=True)

## Just the DNA repair branch of Yeast GO

In [6]:
f = np.load('/cellar/users/mikeyu/DeepCell/go_6oct2017_v2/resnik_ss_GO_0006281.npz')
ss, ss_nodes = f['ss'], f['ss_nodes']

ont = Ontology.from_table('/cellar/users/mikeyu/DeepCell/go_6oct2017_v2/collapsed_go_GO_0006281.propagated', clixo_format=True)

## Fan's HNeXO similarity

In [6]:
f = np.load('/cellar/users/mikeyu/DeepTranslate/hnexo/RFv2r3_square.npz')
ss, ss_nodes = f['rf'], f['genes']

# Iterate

In [7]:
ss = np.round(ss, 2)
np.triu_indices(ss.shape[0], k=1)
ss_flat_i, ss_flat_j = np.triu_indices(ss.shape[0], k=1)
ss_flat = ss[ss_flat_i, ss_flat_j]
print 'Distinct similarity scores:', np.unique(ss_flat).size

Distinct similarity scores: 408


In [8]:
## Initialize data structures
ss_argsort = np.argsort(-1 * ss_flat)

In [9]:
def count_unique_terms(ont, k):
    tmp = set([(tuple(sorted(ont.genes[g] for g in ont.term_2_gene[ont.terms[i]]))) for i in range(len(ont.terms)) if ont.term_sizes[i]==k])
    return len(tmp)

In [10]:
def collapse_identical(ont):
    ont = ont.propagate(gene_term=True, term_term=False, inplace=False)
    first = lambda x: x[0]
    import itertools
    term_groups = [[b for a, b in t_list] for _, t_list in itertools.groupby(sorted([(tuple(sorted(g)), t) for t, g in ont.term_2_gene.items()], key=first), key=first)]
    term_2_group = {t : '|'.join(t_list) for t_list in term_groups for t in t_list}
    return ont.rename(terms=term_2_group)

ont = collapse_identical(ont)

In [11]:
def expand_csr(X, new_shape):
    assert isspmatrix_csr(X)
    X.indptr = np.append(X.indptr, np.repeat(X.indptr[-1], new_shape[0] - X.shape[0]))
    X._shape = new_shape
    return X

In [12]:
def clixov(ss_argsort, n, beta, ont=None):
    dt = ss_flat[ss_argsort[0]]
    val_prev = dt
    dt_iter = 0
    n_genes = ss.shape[0]
    i_list, j_list = [], []

    n = ss.shape[0]
    G = csc_matrix((n,n), dtype=np.float32)
    X = np.zeros((n,n), dtype=np.float32)
    np.fill_diagonal(X, 1)
    X = csc_matrix(X)
    
    H = scipy.sparse.csr_matrix((n,n), dtype=np.bool)
    
    GX = csc_matrix((n,0), dtype=np.float32).toarray()
    prev_X_size = 0

    prev_merge = csr_matrix((n,n), dtype=np.int32)
#     prev_merge_i, prev_merge_j = np.array([], np.int32), np.array([], np.int32)
    
    if ont is not None:
        ont = ont.propagate(gene_term=True, term_term=False, inplace=False)
        term_sizes = np.array(ont.term_sizes)

    for ss_idx in ss_argsort:
        val = ss_flat[ss_idx]

        if val != val_prev or ss_idx==ss_argsort[-1]:
            prev_dt, dt, val_prev = dt, val_prev, val
            if ss_idx == ss_argsort[-1]:
                i, j = ss_flat_i[ss_idx], ss_flat_j[ss_idx]
                i_list.append(i)
                j_list.append(j)
            
            print '###########################################'
            start_clixov = time.time()
            expected_size = int(round(2**(-1 * dt) * n_genes))
            print 'dt_iter:', dt_iter, 'dt:', dt, 'prev_dt:', prev_dt, 'Expected size:', expected_size, round(2**(-1 * dt) * n_genes, 2)
            if ont is not None:
                print 'Ref terms:', (term_sizes==expected_size).sum()                
            print 'Total cliques:', X.shape[1], format_clique_sizes(X)

            dt_iter += 1
            print len(i_list), 'Added edges', G.sum() / 2, 'Current edges'

            # Only add edges that aren't in G yet
            tmp = as_dense_flat(G[i_list, j_list] == 0.)
            i_list, j_list = np.array(i_list)[tmp], np.array(j_list)[tmp]
            print "Edges not already in G:", i_list.size

            dG = coo_matrix((np.ones(i_list.size * 2, G.dtype),
                            (np.append(i_list, j_list), np.append(j_list, i_list))),
                            G.shape, copy=False)
            dG = csc_matrix(dG)
            newG = G + dG

            recent_X = X[:, prev_X_size:]
            recent_GX = as_dense_array(dot(newG, recent_X))
            GX = np.hstack([GX + as_dense_array(dot(dG, X[:,:prev_X_size])), recent_GX])
            
            # Reshape
            expand_csr(prev_merge, (X.shape[1], X.shape[1]))
            
#             if dt_iter==6:
#                 return X, GX, newG, G, dG, prev_merge_i, prev_merge_j, H
            
#             dX, merge_i, merge_j = calc_and_do_merge(X,
            dX, merge = calc_and_do_merge(X,
                                         X.toarray(),
                                         GX,
                                         newG,
                                         beta,
                                         prev_merge,
                                         H=H,
                                         dG=dG)
            assert_unique_cliques(dX)
            prev_merge += merge
            
#             prev_merge_i, prev_merge_j = np.append(prev_merge_i, merge_i), np.append(prev_merge_j, merge_j)
            print 'New cliques formed from merging:', dX.shape[1], format_clique_sizes(dX)
            
            unexplained = get_unexplained_edges(dX, dG)
            unexplained_n = unexplained.sum()
            print 'Unexplained edges:', unexplained_n
            assert unexplained_n == 0
            
            H = csr_matrix(update_subsumption(X, dX, H))
            assert np.all(H.data)
            
            prev_X_size = X.shape[1]
            G = newG

            if unexplained_n > 0:
                augment = clique_maxcover.max_clique_cover(unexplained, newG)
                augment = tuples_to_csc(augment, n, X.dtype)
                print 'Augmenting cliques:', augment.shape[1], format_clique_sizes(augment)
                assert augment.shape[1]>0
                X = csc_matrix(scipy.sparse.hstack([X, dX, augment]))
            else:
                X = csc_matrix(scipy.sparse.hstack([X, dX]))

            print 'prev_X_size:', prev_X_size        
            print 'prev_merge:', prev_merge.sum() / 2
            print 'Total iteration time:', time.time() - start_clixov
            i_list, j_list = [], []

        # Collect the gene pairs that have the same (or similar) similarities
    #     i, j = ss_idx / n_genes, ss_idx % n_genes
        i, j = ss_flat_i[ss_idx], ss_flat_j[ss_idx]
        i_list.append(i)
        j_list.append(j)
        
    return X

In [ ]:
n = ss.shape[0]
beta = 1.0
# X, GX, newG, G, dG, prev_merge_i, prev_merge_j, H = clixov(ss_argsort, n, beta, ont)
# X, GX, newG, G, dG, prev_merge_i, prev_merge_j, unexplained, dX, merge_i, merge_j, augment = clixov(ss_argsort, n, beta)
# X = clixov(ss_argsort, n, beta)

# X = clixov(ss_argsort, n, 1.0)
%lprun -f clixov -f calc_and_do_merge -f get_unexplained_edges -f get_merge_mask1 -f get_thresholds_symm -f subsumption -f do_merge -f get_unexplained_edges -f fill_diagonal X = clixov(ss_argsort, n, 1.0, ont)
# %lprun -f get_merge_mask1 X = clixov(ss_argsort, n, beta, ont)

###########################################
dt_iter: 0 dt: 11.65 prev_dt: 11.65 Expected size: 2 2.01
Ref terms: 675
Total cliques: 6448 OrderedDict([(1, 6448)])
675 Added edges 0.0 Current edges
Edges not already in G: 675
onlytest: 1350
Test 675 out of 41570256 clique pairs:
Test merge time: 0.0196640491486
To merge: 675
get_merge_mask1 time: 0.0229389667511
Iteration: 0 G/dG sum: 0 1350
	Transferred from dG to G: 0
	Search tree nodes / time: 1857 0.00607204437256
	Meta cliques: 675 OrderedDict([(2, 675)])
	Unexplained meta edges: 0
Merge cliques: 675 0.000500917434692 OrderedDict([(2, 675)])
orig_dG_genes: 1350 0.00997495651245
Merge cliques (cover): 675 OrderedDict([(2, 675)])
do_merge time: 0.928247213364
New cliques formed from merging: 675 OrderedDict([(2, 675)])
Unexplained edges: 0.0
prev_X_size: 6448
prev_merge: 675
Total iteration time: 2.17329192162
###########################################
dt_iter: 1 dt: 11.07 prev_dt: 11.65 Expected size: 3 3.0
Ref terms: 466
Total cliq

orig_dG_genes: 6496 0.0305218696594
Merge cliques (cover): 146 OrderedDict([(9, 145), (10, 1)])
do_merge time: 0.096951007843
New cliques formed from merging: 146 OrderedDict([(9, 145), (10, 1)])
Unexplained edges: 0.0
prev_X_size: 8827
prev_merge: 23845
Total iteration time: 0.521633148193
###########################################
dt_iter: 8 dt: 9.33 prev_dt: 9.48 Expected size: 10 10.02
Ref terms: 127
Total cliques: 8973 OrderedDict([(1, 6448), (2, 675), (3, 469), (4, 366), (5, 266), (6, 235), (7, 198), (8, 170), (9, 145), (10, 1)])
3171 Added edges 16713.0 Current edges
Edges not already in G: 3171
onlytest: 88994
Test 44497 out of 80505756 clique pairs:
Test merge time: 0.110267877579
To merge: 5203
get_merge_mask1 time: 0.115628004074
Iteration: 0 G/dG sum: 74334 10406
	Transferred from dG to G: 2598
	Search tree nodes / time: 3566 0.0284759998322
	Meta cliques: 204 OrderedDict([(2, 52), (3, 58), (4, 27), (5, 19), (6, 9), (7, 12), (8, 10), (9, 6), (10, 11)])
	Unexplained meta ed

Total iteration time: 0.615392923355
###########################################
dt_iter: 14 dt: 8.65 prev_dt: 8.75 Expected size: 16 16.05
Ref terms: 73
Total cliques: 9577 OrderedDict([(1, 6448), (2, 675), (3, 469), (4, 366), (5, 266), (6, 235), (7, 198), (8, 170), (9, 145), (10, 125), (11, 122), (12, 109), (13, 97), (14, 77), (15, 74), (16, 1)])
4586 Added edges 39171.0 Current edges
Edges not already in G: 4586
onlytest: 207452
Test 103726 out of 91709352 clique pairs:
Test merge time: 0.151471138
To merge: 7352
get_merge_mask1 time: 0.159248113632
Iteration: 0 G/dG sum: 167414 14704
	Transferred from dG to G: 4150
	Search tree nodes / time: 6319 0.0552499294281
	Meta cliques: 398 OrderedDict([(2, 36), (3, 76), (4, 75), (5, 70), (6, 48), (7, 30), (8, 24), (9, 10), (10, 6), (11, 1), (12, 5), (13, 3), (14, 4), (15, 6), (16, 4)])
	Unexplained meta edges: 2
Iteration: 1 G/dG sum: 182116 2
	Transferred from dG to G: 0
	Search tree nodes / time: 4 0.0208790302277
	Meta cliques: 1 Ordered

onlytest: 217756
Test 108878 out of 98000100 clique pairs:
Test merge time: 0.127006053925
To merge: 6255
get_merge_mask1 time: 0.134771823883
Iteration: 0 G/dG sum: 261110 12510
	Transferred from dG to G: 5098
	Search tree nodes / time: 3933 0.0527908802032
	Meta cliques: 315 OrderedDict([(2, 24), (3, 45), (4, 64), (5, 60), (6, 40), (7, 27), (8, 18), (9, 11), (10, 15), (11, 4), (12, 1), (16, 1), (18, 2), (20, 2), (21, 1)])
	Unexplained meta edges: 24
Iteration: 1 G/dG sum: 273596 24
	Transferred from dG to G: 2
	Search tree nodes / time: 46 0.0267810821533
	Meta cliques: 11 OrderedDict([(3, 2), (4, 3), (5, 2), (6, 3), (7, 1)])
	Unexplained meta edges: 0
Merge cliques: 326 0.000643968582153 OrderedDict([(3, 14), (4, 23), (5, 41), (6, 34), (7, 38), (8, 40), (9, 28), (10, 25), (11, 19), (12, 10), (13, 4), (14, 4), (15, 5), (16, 1), (17, 2), (21, 38)])
orig_dG_genes: 6972 0.0325469970703
Merge cliques (cover): 38 OrderedDict([(21, 38)])
do_merge time: 0.158957004547
New cliques formed fro

onlytest: 295920
Test 147960 out of 101636642 clique pairs:
Test merge time: 0.158805847168
To merge: 7499
get_merge_mask1 time: 0.168892860413
Iteration: 0 G/dG sum: 350694 14998
	Transferred from dG to G: 6706
	Search tree nodes / time: 9495 0.148842811584
	Meta cliques: 755 OrderedDict([(2, 15), (3, 30), (4, 65), (5, 92), (6, 131), (7, 117), (8, 66), (9, 60), (10, 59), (11, 33), (12, 34), (13, 17), (14, 14), (15, 6), (16, 8), (17, 3), (18, 1), (20, 2), (21, 1), (22, 1)])
	Unexplained meta edges: 8
Iteration: 1 G/dG sum: 365684 8
	Transferred from dG to G: 0
	Search tree nodes / time: 28 0.0411779880524
	Meta cliques: 7 OrderedDict([(4, 1), (5, 5), (11, 1)])
	Unexplained meta edges: 0
Merge cliques: 762 0.000805139541626 OrderedDict([(3, 4), (4, 25), (5, 24), (6, 76), (7, 93), (8, 83), (9, 77), (10, 70), (11, 61), (12, 40), (13, 41), (14, 27), (15, 31), (16, 17), (17, 19), (18, 9), (19, 10), (20, 2), (22, 6), (23, 8), (24, 5), (25, 6), (26, 28)])
orig_dG_genes: 7758 0.0490629673004
M

...

Merge cliques: 190103 0.0687539577484 OrderedDict([(4, 14), (5, 98), (6, 321), (7, 678), (8, 1184), (9, 1999), (10, 2967), (11, 3822), (12, 4600), (13, 5229), (14, 6155), (15, 6830), (16, 7778), (17, 8617), (18, 9359), (19, 9836), (20, 10136), (21, 10501), (22, 10439), (23, 9718), (24, 8843), (25, 8189), (26, 7787), (27, 7919), (28, 7420), (29, 6824), (30, 5928), (31, 4721), (32, 4066), (33, 3469), (34, 2789), (35, 2405), (36, 1987), (37, 1426), (38, 1084), (39, 882), (40, 630), (41, 515), (42, 415), (43, 403), (44, 289), (45, 231), (46, 240), (47, 204), (48, 173), (49, 117), (50, 93), (51, 74), (52, 70), (53, 39), (54, 54), (55, 31), (56, 31), (57, 43), (58, 37), (59, 35), (60, 45), (61, 32), (62, 29), (63, 33), (64, 16), (65, 29), (66, 24), (67, 17), (68, 20), (69, 13), (70, 10), (71, 21), (72, 10), (73, 7), (74, 7), (75, 1), (76, 1), (77, 1), (78, 1), (80, 1), (82, 1), (83, 1), (84, 8), (85, 6), (86, 6), (87, 5), (89, 1), (90, 2), (91, 1), (92, 1), (93, 2), (96, 1), (100, 1), (104, 

Merge cliques: 976295 0.527307033539 OrderedDict([(3, 1), (4, 19), (5, 77), (6, 204), (7, 450), (8, 700), (9, 1126), (10, 2144), (11, 3822), (12, 5935), (13, 9156), (14, 13555), (15, 19330), (16, 26203), (17, 33494), (18, 42226), (19, 48938), (20, 54139), (21, 57090), (22, 59273), (23, 60024), (24, 57756), (25, 54837), (26, 50351), (27, 46786), (28, 41481), (29, 37078), (30, 31615), (31, 27366), (32, 23091), (33, 19465), (34, 16993), (35, 15225), (36, 13728), (37, 11667), (38, 10401), (39, 9111), (40, 7793), (41, 6837), (42, 5866), (43, 5442), (44, 5129), (45, 4681), (46, 4127), (47, 3777), (48, 3335), (49, 2977), (50, 2334), (51, 2018), (52, 1743), (53, 1512), (54, 1201), (55, 1165), (56, 1060), (57, 1141), (58, 1213), (59, 1245), (60, 1159), (61, 995), (62, 955), (63, 760), (64, 643), (65, 472), (66, 301), (67, 291), (68, 221), (69, 153), (70, 116), (71, 122), (72, 121), (73, 79), (74, 96), (75, 72), (76, 65), (77, 52), (78, 47), (79, 24), (80, 22), (81, 15), (82, 7), (83, 6), (84, 9

	Unexplained meta edges: 30
Iteration: 1 G/dG sum: 2667382 30
	Transferred from dG to G: 0
	Search tree nodes / time: 17850 0.148714065552
	Meta cliques: 4664 OrderedDict([(5, 2), (6, 4), (7, 18), (8, 41), (9, 71), (10, 171), (11, 366), (12, 728), (13, 884), (14, 762), (15, 596), (16, 443), (17, 237), (18, 143), (19, 98), (20, 42), (21, 20), (22, 13), (23, 9), (24, 2), (26, 3), (27, 1), (28, 1), (30, 1), (31, 2), (34, 1), (37, 1), (38, 1), (39, 2), (40, 1)])
	Unexplained meta edges: 0
Merge cliques: 205745 0.128859043121 OrderedDict([(4, 5), (5, 15), (6, 39), (7, 92), (8, 83), (9, 141), (10, 173), (11, 214), (12, 397), (13, 838), (14, 1316), (15, 1870), (16, 2389), (17, 2650), (18, 3006), (19, 3849), (20, 4080), (21, 4119), (22, 4769), (23, 5400), (24, 5956), (25, 7099), (26, 7772), (27, 8458), (28, 8925), (29, 9224), (30, 8979), (31, 8636), (32, 8467), (33, 7658), (34, 7365), (35, 6957), (36, 6659), (37, 6039), (38, 5430), (39, 5175), (40, 5000), (41, 4528), (42, 4187), (43, 3942), (4

Merge cliques: 224672 0.127570867538 OrderedDict([(7, 2), (8, 8), (9, 16), (10, 71), (11, 225), (12, 424), (13, 937), (14, 1578), (15, 2610), (16, 4082), (17, 5648), (18, 7179), (19, 8569), (20, 9389), (21, 9925), (22, 10187), (23, 11288), (24, 11421), (25, 11134), (26, 10825), (27, 10341), (28, 10137), (29, 9747), (30, 9199), (31, 8212), (32, 7015), (33, 5945), (34, 5560), (35, 5283), (36, 4630), (37, 3981), (38, 3155), (39, 2711), (40, 2392), (41, 2267), (42, 2039), (43, 1917), (44, 2007), (45, 1718), (46, 1583), (47, 1402), (48, 1325), (49, 1344), (50, 1039), (51, 1036), (52, 885), (53, 777), (54, 755), (55, 788), (56, 749), (57, 615), (58, 570), (59, 652), (60, 592), (61, 623), (62, 699), (63, 664), (64, 611), (65, 617), (66, 533), (67, 483), (68, 382), (69, 337), (70, 298), (71, 275), (72, 214), (73, 147), (74, 118), (75, 103), (76, 64), (77, 75), (78, 51), (79, 50), (80, 41), (81, 47), (82, 51), (83, 46), (84, 38), (85, 50), (86, 27), (87, 25), (88, 16), (89, 10), (90, 8), (91, 1

Merge cliques: 565203 0.343858003616 OrderedDict([(4, 24), (5, 117), (6, 460), (7, 1084), (8, 1953), (9, 2976), (10, 4115), (11, 5547), (12, 7361), (13, 8965), (14, 10680), (15, 12262), (16, 13392), (17, 14408), (18, 14774), (19, 15015), (20, 15768), (21, 15776), (22, 15360), (23, 16347), (24, 16233), (25, 17395), (26, 18476), (27, 19792), (28, 20303), (29, 20528), (30, 21098), (31, 20586), (32, 20379), (33, 19840), (34, 19442), (35, 19073), (36, 18324), (37, 16821), (38, 15153), (39, 13505), (40, 11685), (41, 10563), (42, 8736), (43, 7680), (44, 6614), (45, 5507), (46, 5048), (47, 4319), (48, 4062), (49, 3615), (50, 3086), (51, 2634), (52, 2113), (53, 1927), (54, 1707), (55, 1295), (56, 1257), (57, 1058), (58, 849), (59, 756), (60, 603), (61, 597), (62, 505), (63, 536), (64, 578), (65, 484), (66, 498), (67, 453), (68, 446), (69, 409), (70, 299), (71, 273), (72, 261), (73, 212), (74, 214), (75, 207), (76, 176), (77, 152), (78, 86), (79, 68), (80, 73), (81, 45), (82, 37), (83, 37), (84,

In [14]:
with open('tmp.npy') as f:
    X, dG = cPickle.load(f)

In [ ]:
import networkx as nx
import pygraphviz
from networkx.drawing.nx_agraph import graphviz_layout

In [96]:
# H = csr_matrix(subsumption(X))
# # H = csr_matrix(subsumption(X[:,[7, 6448]]))
# import igraph
# g = igraph.Graph(n=H.shape[0], edges=zip(*H.nonzero()), directed=True)
# assert g.is_dag()
# topo = g.topological_sorting(mode='out')
# print [(e.source, e.target) for e in g.es[:5]]
# # print list(topo)
# print topo.index(7), topo.index(6448)

# # import clique_maximal
# # clique_maximal.BK_hier_Gsep_wrapper(Gold, Gnew, H)

In [16]:
A, GA, G = X.toarray(), GX, newG
# A, GA, G = X.toarray()[:,[345, 703]], GX[:,[345, 703]], newG

threshold_loose, threshold_strict = get_thresholds(A, A, beta)
i_merge, j_merge = get_merge_mask1(A, GA, A, GA, G,
                                   threshold_loose, threshold_strict,
                                   symm=True,
                                   notest=(prev_merge_i, prev_merge_j))
# i_merge, j_merge = get_merge_mask1(A, GA, A, GA, G,
#                                    threshold_loose, threshold_strict,
#                                    symm=True)

Test 1044336 out of 25365003 clique pairs:
Merge time: 1.99702906609
To merge: 2755


In [ ]:
merge = do_merge(X, i_merge, j_merge, prev_merge_i, prev_merge_j)

# max_size = 0
# for i in range(merge.shape[1]):
#     clique = tuple(sorted(merge[:,i].nonzero()[0]))
#     max_size = max(max_size, len(clique))
#     if len(clique)>=4:
#         print '*', as_dense_array(G[clique,:][:,clique])
#     assert_clique(clique, G)
# print 'max_size:', max_size

In [54]:
newX, merge_i, merge_j = calc_and_do_merge(X,
                                           X.toarray(),
                                           GX,
                                           newG,
                                           beta,
                                           prev_merge_i,
                                           prev_merge_j,
                                           H=H,
                                           dG=dG)

onlytest: 416
Test 197 out of 106602 clique pairs:
Test merge time: 0.000344038009644
To merge: 113
get_merge_mask1 time: 0.00427007675171
Iteration: 0 G/dG sum: 448 226
[232 286]
[ 15 170 232]
	Transferred from dG to G: 60
	Search tree nodes / time: 74 0.0268878936768
	Meta cliques: 3 OrderedDict([(5, 2), (6, 1)])
	Unexplained meta edges: 0
Merge cliques: 3 0.000420093536377 OrderedDict([(8, 3)])
orig_dG_genes: 116 0.00335502624512
Merge cliques (cover): 3 OrderedDict([(8, 3)])
do_merge time: 0.0528020858765


# asdf

In [ ]:
with open('tmp.npy', 'wb') as f: cPickle.dump((Gold, Gnew, H), f)

In [102]:
np.all(subsumption(X).toarray() == H.toarray())

False

In [111]:
print H[310,:].nonzero()[1]
print H[315,:].nonzero()[1]

[]
[]


In [14]:
with open('tmp.npy') as f:
    Gold, Gnew, H = cPickle.load(f)

In [75]:
C, CP, CN, tree = clique_maximal.BK_dG_py(Gold, Gnew)
cliques_csc = cliques_to_csc(C, CP, CN, n)
print cliques_csc.shape
for i in range(cliques_csc.shape[1]):
    print cliques_csc[:,i].nonzero()[0]

(304, 3)
[  0 104 136 153 221 232 257 286]
[ 17 128 138 148 231 233 249 270]
[ 17  54 147 162 189 196 245 274]


In [76]:
[g for t, g in ont.term_2_gene.items() if len(g)==8]

[[0, 104, 136, 153, 221, 232, 257, 286],
 [17, 128, 138, 148, 231, 233, 249, 270],
 [17, 54, 147, 162, 189, 196, 245, 274]]

In [ ]:
C, CP, CN, tree = clique_maximal.BK_hier_dG_py(Gold, Gnew, H)
cliques_csc = cliques_to_csc(C, CP, CN, n)
print cliques_csc.shape

In [74]:
Gold, Gnew = clixov_utils.get_test_new_network('cluster', k=200, r=10, s=10, verbose=False)
%time cliques, cliques_indptr, cliques_n, tree_size = clique_maximal.BKPivotSparse2_Gnew_wrapper(Gold, Gnew)
cliques = cliques_to_csc(cliques, cliques_indptr, cliques_n, Gold.shape[0])

tmp1 = csc_to_cliques_list(cliques)
%time tmp2 = clique_maximal.get_cliques_igraph(Gold.shape[0], Gold + Gnew, Gnew, input_fmt='matrix')
assert set(tmp1) == set(tmp2)
len(tmp1)

CPU times: user 0 ns, sys: 4 ms, total: 4 ms
Wall time: 2.45 ms
CPU times: user 152 ms, sys: 0 ns, total: 152 ms
Wall time: 149 ms


37

In [145]:
X_set = set([])
for i in as_dense_flat(X.sum(0)==4).nonzero()[0]:
    X_set.add(tuple(sorted(X[:, i].nonzero()[0])))
len(X_set)

386

In [144]:
ont_set = set([tuple(sorted(ont.term_2_gene[t])) for t, s in zip(ont.terms, ont.term_sizes) if s==4])
len(ont_set)

303